## Microsoft Defender for External Attack Surface Management (MDEASM) Python Jupyter Notebook
***
This is a sample notebook for querying your MDEASM inventory. A few prerequisites must be completed before you can use it properly:
1. You must complete an initial __[Discovery](https://learn.microsoft.com/en-us/azure/external-attack-surface-management/using-and-managing-discovery)__ in your MDEASM workspace to which you have, at a minimum, _read_ permissions
2. You will also need your _Subscription Id_, _Tenant Id_, _Resource Group Name_, _Workspace Name_, & the Azure _Region_ your workspace was initialized in
3. You must have a registered __[Service Principal](https://learn.microsoft.com/en-us/rest/api/defenderforeasm/authentication#client-service-principal)__ application that is properly configured to interact with our REST API
4. To create a bearer token with your _Service Principal_ you will need the app _Client Id_, _Client Secret_, & _Tenant Id_ 
5. A minimum of a 3.X Python installation with an installed Jupyter module OR and IDE such as VSCode with a Jupyter Notebook extension installed
***

> Note for simplicity's sake, we are putting the auth variables here in the notebook for demo purposes only. In reality, this should __never__ take place in a document/script you share with others, or publish online. Storage of clientIds and clientSecrets should always leverage a more secure solution such as Key Vaults or encrypted storage of some kind. If you share this notebook with others after usage, clear outputs of all cells and restart your kernel after you have deleted these sensetive variables from your code.

In [ ]:
import requests, time, json, re

# Important variables to set
bearerToken = None # This will be set by the azure_auth() function otherwise enter the token as a string in side quotes ''
bearerTokenExpires = 0 # This will be set by the azure_auth() function otherwise enter the epoch time in seconds as an integer
clientId = '' # ex. 'a8c5a9e0-0000-0000-0000-000000000000' => Application (client) ID - string [required]
clientSecret = '' # ex. 'ABCDE~123456789abcdefg_9876543210~ZyXwVu' => Application (client) secret - string [required]
tenantId = '' # ex. 'a8c5a9e0-0000-0000-0000-000000000000' => Directory (tenant) ID - string [required]
subscriptionId = '' # ex. 'a8c5a9e0-0000-0000-0000-000000000000' => Subscription ID - string [required]
resourceGroupName = '' # ex. 'myMDEASMAPItest-rg' => Resource group name - string [required]
resourceName = '' # ex. 'myMDEASMworkspace' => Resource/Workspace name - string [required]
region = '' # ex. 'eastus' => Region - string [required]
# Leave these variables unchanged until you need to change them
resource = 'https://easm.defender.microsoft.com/' # => API Resource - string [supplied]
apiVersion = '2022-11-01-preview' # => API Version - string [supplied]
maxpagesize = 25 # => Max page size - int [required]
mark = '*' # => Mark - string [optional]
postBody = '' # => Post body - JSON formatted string [optional]
# Utilitiy variables
uuidRegex = '[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}'
encodingRegex = "[\s\!\@\#\$\^\*\(\)\_\=\-\[\]\{\}\;\:\'\"\,\.\<\>\/\?\|\\\]"
results = set({})
nextUrl = None
planeType = 'data' # => Plane type - string [required] - 'data' or 'control'
currentPlane = 'data'

# Function to take a filter and check if it is URL encoded, if not, then encode it
def check_url_encoding(filter):
    if filter != '' and re.search(encodingRegex, filter):
        query = requests.utils.quote(filter, safe='+')
    else:
        query = filter
    return query

# Helper Function to authenticate and obtain a bearer token to be run before every request
def azure_auth():
    # Check if required variables are set
    global bearerToken, bearerTokenExpires, planeType, currentPlane
    planeType = planeType.lower()
    if planeType == 'data':
        resource = 'scope=https%3A%2F%2Feasm.defender.microsoft.com%2F.default'
    elif planeType == 'control':
        resource = 'scope=https%3A%2F%2Fmanagement.azure.com%2F.default'
    else:
        print("Invalid plane type must be either 'data' or 'control'")
        return None

    if clientId == None or clientSecret == None or tenantId == None:
        print("Please provide a client ID, client secret, and tenant ID or if you acquired your bearer token via the CLI please enter it in the bearerToken and bearerTokenExpires parameters above.")
        return None
    # Check if bearer token is set and not expired, else return a new token
    if bearerToken == None or bearerToken == '' and bearerTokenExpires == 0 or time.time() > bearerTokenExpires or currentPlane != planeType or currentPlane == None:
        auth_url = f"https://login.microsoftonline.com/{tenantId}/oauth2/v2.0/token"
        auth_payload=f'grant_type=client_credentials&client_id={clientId}&client_secret={clientSecret}&{resource}'
        auth_headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        response = requests.request("POST", auth_url, headers=auth_headers, data=auth_payload)
        if response.status_code != 200:
            print("Error acquiring bearer token")
            print(response.text)
            return None
        else:
            bearerToken = json.loads(response.text)['access_token']
            bearerTokenExpires = time.time() + json.loads(response.text)['expires_in']
            currentPlane = planeType
    return bearerToken, bearerTokenExpires

# Make an initial request for a bearer token
azure_auth()
if bearerToken != None and bearerTokenExpires == 0 or time.time() > bearerTokenExpires:
    print("Error acquiring bearer token")
else:
    print("Bearer token acquired successfully")

### Assets - List

In [ ]:
## Assets List returns##
def get_asset_list(query):
    ''' Assets List endpoint - takes a URL encoded query string '''
    check_url_encoding(query)
    global planeType
    planeType = 'data'
    azure_auth()
    nextUrl = None
    url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/assets?api-version={apiVersion}&maxpagesize={maxpagesize}&filter={query}&mark={mark}"

    payload={}
    headers = {
      'User-Agent': 'MDEASM Python Notebook',
      'Authorization': f'Bearer {bearerToken}'
    }
    

    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code != 200:
        print("Error getting asset list")
        print(response.text)
        return None
    if len(response.text) == 0:
        print("No assets found matching your query")
        return None
    else:
        responsejson = json.loads(response.text)
        responseresults = responsejson['content']
        for i in responseresults:
            results.add(i['name'])
        if 'nextLink' in responsejson and nextUrl != responsejson['nextLink']:
            nextUrl = responsejson['nextLink']
        else:
            nextUrl = None
        #print(responsejson)
        
    while nextUrl:
        azure_auth()
        response = requests.request("GET", nextUrl, headers=headers, data=payload)
        responsejson = json.loads(response.text)
        responseresults = responsejson['content']
        for i in responseresults:
            results.add(i['name'])
        if 'nextLink' not in responsejson:
            nextUrl = None
        else:
            time.sleep(1)
            nextUrl = responsejson['nextLink']
        
    if len(results) == 0:
        print("No assets found matching your query")
    else:
        print("Number of results: " + str(len(results)))
    print(results)

In [ ]:
query = 'state = confirmed AND kind in ("host","ipAddress") AND webComponentType = Server AND webComponentName ^=in ("Apache", "Microsoft IIS") AND cvss3BaseScore >= 7'
get_asset_list(query)

### Asset - Get

In [ ]:
# Retrieves a single asset by assetId
def get_asset(assetId):
    ''' Asset Get endpoint - takes a UUID formatted AssetId '''
    if assetId!= None and re.search(uuidRegex, assetId) != None:
        global planeType
        planeType = 'data'
        azure_auth()
        url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/assets/{assetId}?api-version={apiVersion}"
        payload={}
        headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}'
        }
        response = requests.request("GET", url, headers=headers, data=payload)
        if response.status_code != 200:
            print("Error getting asset details")
            print(response.text)
            return None
        if len(response.text) == 0:
            print("No asset found matching your assetId")
            return None
        else:
            responsejson = json.loads(response.text)
            
            # simplified output of asset names, adjust to your needs
            print(responsejson)
    else:
        print("Please provide a valid assetId")

In [ ]:
get_asset('a8c5a9e0-0000-0000-0000-000000000000')

### Assets - Update

In [ ]:
# Update labels, state, or externalId for a set of assets returned by a query
def update_assets(query, postBody):
    ''' Function to access the Assets Update endpoint - requires a URL encoded query string 
    and a post body containing the asset properties to update 
    example post body: {"labels": {"Label1": True}, "state": "confirmed", "transfers":["sslCert"], "externalId": "123456"}
    accepted transfers values: contact, domain, ipBlock, host, page, sslCert, as, ipAddress'''
    check_url_encoding(query)
    global planeType
    planeType = 'data'
    azure_auth()
    url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/assets?filter={query}&api-version={apiVersion}"
    payload = json.loads(json.dumps(postBody))
    headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.request("PATCH", url, headers=headers, data=payload)
    if response.status_code != 200:
        print("Error updating assets related to supplied query")
        print(response.text)
        return None
    else:
        responsejson = json.loads(response.text)
        print(responsejson)

In [ ]:
update_query = 'state = confirmed AND domain in ("example.com")'
update_body = '{"state":"dismissed","transfers":["sslCert"]}'
update_assets(update_query, update_body)

### Tasks - Get

In [ ]:
# Retrieves a single task by taskId
def get_task(taskId):
    ''' Task Get endpoint - requires a UUID formatted TaskId '''
    if taskId!= None and re.search(uuidRegex, assetId) != None:
        global planeType
        planeType = 'data'
        azure_auth()
        url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/tasks/{taskId}?api-version={apiVersion}"
        headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}'
        }
        response = requests.request("GET", url, headers=headers)
        if response.status_code != 200:
            print("Error getting task details")
            print(response.text)
            return None
        if len(response.text) == 0:
            print("No task found matching your taskId")
            return None
        else:
            responsejson = json.loads(response.text)
            
            # simplified output of asset names, adjust to your needs
            print(responsejson)
    else:
        print("Please provide a valid taskId")

In [ ]:
get_task('a8c5a9e0-0000-0000-0000-000000000000')

### Tasks - List

In [ ]:
# Retrieves a list of tasks by workspace name
def get_task_list():
    ''' Tasks List endpoint - requires a valid resource name (the name of your workspace) '''
    global planeType
    planeType = 'data'
    azure_auth()
    url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/tasks?api-version={apiVersion}"
    headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}'
    }
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        print("Error getting task list, check the resource name")
        print(response.text)
        return None
    if len(response.text) == 0:
        print("No tasks found matching your resource name")
        return None
    else:
        responsejson = json.loads(response.text)
        
        # simplified output of workspace tasks, adjust to your needs
        print(responsejson)

In [ ]:
get_task_list()

### Tasks - Cancel

In [ ]:
# Cancels a running task by taskId
def cancel_task(taskId):
    ''' Task Cancel endpoint - requires a UUID formatted TaskId '''
    if taskId!= None and re.search(taskId) != None:
        global planeType
        planeType = 'data'
        azure_auth()
        url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/tasks/{taskId}:cancel?api-version={apiVersion}"
        headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}'
        }
        response = requests.request("GET", url, headers=headers)
        if response.status_code != 200:
            print("Error getting task details")
            print(response.text)
            return None
        if len(response.text) == 0:
            print("No task found matching your taskId")
            return None
        else:
            responsejson = json.loads(response.text)
            
            # simplified output of asset names, adjust to your needs
            print(responsejson)
    else:
        print("Please provide a valid taskId")

In [ ]:
cancel_task('a8c5a9e0-0000-0000-0000-000000000000')

### Helper functions:
### Get Common Assets

In [ ]:
# Function to gather all the common assets for a given query 
# Eample: all domains for a host based query - query=query and the type="host"
def get_common_assets(query):
    check_url_encoding(query)
    global planeType, nextUrl
    planeType = 'data'
    azure_auth()
    nextUrl = None
    commonAssetsList = []
    def commonFunc(assetName):
        if any(assetName in x.values() for x in commonAssetsList):
            for i in commonAssetsList:
                if assetName == str(i['name']):
                    i['count'] += 1
        else:
            commonAssetsList.append({"name": str(assetName), "count": 1})

    url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/assets?api-version={apiVersion}&maxpagesize={maxpagesize}&filter={query}&mark={mark}"

    payload={}
    headers = {
      'User-Agent': 'MDEASM Python Notebook',
      'Authorization': f'Bearer {bearerToken}'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code != 200:
        print("Error getting asset list")
        print(response.text)
        return None
    if len(response.text) == 0:
        print("No assets found matching your query")
        return None
    else:
        responsejson = json.loads(response.text)
        responseresults = responsejson['content']   
        for i in responseresults:
            if i['kind'] == 'host' or i['kind'] == 'page':
                commonAssetName = str(i['asset']['domain'])
                commonFunc(commonAssetName)
            elif i['kind'] == 'ipAddress':
                try:
                    if len(i['asset']['ipBlocks']) > 0:
                        for x in i['asset']['ipBlocks']:
                            commonAssetName = str(x['ipBlock'])
                            commonFunc(commonAssetName)
                except:
                    pass
                try:
                    if len(i['asset']['asns']) > 0:
                        for x in i['asset']['asns']:
                            commonAssetName = str(x['value'])
                            commonFunc(commonAssetName)
                except:
                    pass
            elif i['kind'] == 'ipBlock':
                try:
                    if len(i['asset']['ipBlocks']) > 0:
                        for x in i['asset']['ipBlocks']:
                            commonAssetName = str(x['ipBlock'])
                            commonFunc(commonAssetName)
                except:
                    pass
            else:
                pass

        if 'nextLink' in responsejson and nextUrl != responsejson['nextLink']:
            nextUrl = responsejson['nextLink']
        else:
            nextUrl = None
        
    while nextUrl:
        azure_auth()
        time.sleep(1)
        response = requests.request("GET", nextUrl, headers=headers, data=payload)
        responsejson = json.loads(response.text)
        responseresults = responsejson['content']
        for i in responseresults:
            if i['kind'] == 'host' or i['kind'] == 'page':
                commonAssetName = str(i['asset']['domain'])
                commonFunc(commonAssetName)
            elif i['kind'] == 'ipAddress':
                try:
                    if len(i['asset']['ipBlocks']) > 0:
                        for x in i['asset']['ipBlocks']:
                            commonAssetName = str(x['ipBlock'])
                            commonFunc(commonAssetName)
                except:
                    pass
                try:
                    if len(i['asset']['asns']) > 0:
                        for x in i['asset']['asns']:
                            commonAssetName = str(x['value'])
                            commonFunc(commonAssetName)
                except:
                    pass
            elif i['kind'] == 'ipBlock':
                try:
                    if len(i['asset']['ipBlocks']) > 0:
                        for x in i['asset']['ipBlocks']:
                            commonAssetName = str(x['ipBlock'])
                            commonFunc(commonAssetName)
                except:
                    pass
            else:
                pass

        if 'nextLink' not in responsejson:
            nextUrl = None
        else:
            nextUrl = responsejson['nextLink']   
    commonAssetsList = sorted(commonAssetsList, key=lambda k: k['count'], reverse=True)
    for i in commonAssetsList:
        print(str(i))


In [ ]:
query = 'state = confirmed AND kind in ("host","ipBlock")'
get_common_assets(query)

### Control Plane Endpoints
### Labels - Create And Update

In [ ]:
# Labels Create And Update endpoint - requires a URL encoded query string
#   and a post body containing the label properties to create or update 
#   example labelBody: '{"properties": {"color": "red", "displayName": "This is a test label - #1"}}'

def create_labels(labelName, labelBody):
    ''' Labels Create And Update endpoint - requires a URL encoded query string 
    and a post body containing the label properties to create or update 
    example labelBody: {"properties": {"color": "red", "displayName": "This is a test label - #1"}} '''
    labelName = labelName.replace(' ', '')
    global planeType
    planeType = 'control'
    azure_auth()
    url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/labels/{labelName}?api-version=2022-04-01-preview"
    payload = json.loads(json.dumps(labelBody))
    headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.request("PUT", url, headers=headers, data=payload)
    if response.status_code != 200:
        print("Error creating label")
        print(response.text)
        return None
    else:
        responsejson = json.loads(response.text)
        print(responsejson)

In [ ]:
create_labels('TestLabel1', '{"properties": {"color": "red", "displayName": "This is a test label - #1"}}')

### Labels - List By Workspace

In [ ]:
# Returns a list of labels in a workspace
def list_labels():
    ''' Returns a list of labels in a workspace '''
    url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/labels?api-version=2022-04-01-preview"
    global planeType
    planeType = 'control'
    azure_auth()
    headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}'
    }
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        print("Error getting labels list, check the resource name")
        print(response.text)
        return None
    else:
        responsejson = json.loads(response.text)
        print(responsejson)

In [ ]:
list_labels()

### Labels - Get

In [ ]:
# Returns a label in a workspace
def get_label(labelName):
    ''' Returns a label in a workspace '''
    labelName = labelName.replace(' ', '')
    global planeType
    planeType = 'control'
    azure_auth()
    url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/labels/{labelName}?api-version=2022-04-01-preview"
    headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}'
    }
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        print("Error getting labels list, check the resource name")
        print(response.text)
        return None
    else:
        responsejson = json.loads(response.text)
        print(responsejson)

In [ ]:
get_label('TestLabel1')

### Labels - Delete

In [ ]:
# Deletes a label in a workspace
def delete_label(labelName):
    ''' Deletes a label in a workspace '''
    labelName = labelName.replace(' ', '')
    global planeType
    planeType = 'control'
    azure_auth()
    url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/labels/{labelName}?api-version=2022-04-01-preview"
    headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}'
    }
    response = requests.request("DELETE", url, headers=headers)
    responsejson = json.loads(response.text)
    print(responsejson)

In [ ]:
delete_label('TestLabel1')

### Labels - Update

In [ ]:
# Labels Update endpoint - requires a URL encoded query string
#   and a post body containing the label properties to update 
#   example labelBody: '{"properties": {"color": "blue", "displayName": "This is a test label - #2"}}'

def update_labels(labelName, labelBody):
    ''' Labels Update endpoint - requires a URL encoded query string 
    and a post body containing the label properties to update 
    example labelBody: {"properties": {"color": "blue", "displayName": "This is a test label - #2"}} '''
    labelName = labelName.replace(' ', '')
    global planeType
    planeType = 'control'
    azure_auth()
    url = f"https://{region}.easm.defender.microsoft.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/workspaces/{resourceName}/labels/{labelName}?api-version=2022-04-01-preview"
    payload = json.loads(json.dumps(labelBody))
    headers = {
        'User-Agent': 'MDEASM Python Notebook',
        'Authorization': f'Bearer {bearerToken}',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.request("PATCH", url, headers=headers, data=payload)
    if response.status_code != 200:
        print("Error creating label")
        print(response.text)
        return None
    else:
        responsejson = json.loads(response.text)
        print(responsejson)

In [ ]:
update_labels('TestLabel1', '{"properties": {"color": "blue", "displayName": "This is a test label - #2"}}')